In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

In [2]:
train_data = pd.read_json('train_data.json',orient="index")
test_data = pd.read_json('test_data.json',orient='index')

In [3]:
#set index
train_data.reset_index(level = 0, inplace = True)
train_data.rename(columns={'index':'ID'}, inplace=True)

test_data.reset_index(level = 0, inplace = True)
test_data.rename(columns={'index':'ID'}, inplace=True)

In [4]:
#check data
print ('Train data has {} rows and {} columns'.format(train_data.shape[0],train_data.shape[1]))
print ('test_data data has {} rows and {} columns'.format(test_data.shape[0],test_data.shape[1]))

Train data has 200000 rows and 7 columns
test_data data has 100000 rows and 6 columns


In [5]:
#Encode Target Variable
train_data = train_data.replace({'segment':{'pos':1,'neg':0}})

In [6]:
#check target variable count
train_data['segment'].value_counts()/train_data.shape[0]

0    0.923725
1    0.076275
Name: segment, dtype: float64

In [7]:
# Creating separate columns for genres, dow and tod variables

genre_data = [[str("genre_" + i) for i in re.sub(pattern='\:\d+',repl='',string=x).split(",")] for x in train_data['genres']]
dow_data = [[str("dow_" + i) for i in re.sub(pattern='\:\d+', repl='', string = x).split(',')] for x in train_data['dow']]
tod_data = [[str("tod_" + i) for i in re.sub(pattern='\:\d+', repl='', string = x).split(',')] for x in train_data['tod']]
cities_data = [[str("cities_" + i) for i in re.sub(pattern='\:\d+', repl='', string = x).split(',')] for x in train_data['cities']]

genre_data = frozenset.union(*pd.Series(genre_data).apply(frozenset))
dow_data = frozenset.union(*pd.Series(dow_data).apply(frozenset))
tod_data = frozenset.union(*pd.Series(tod_data).apply(frozenset))
cities_data = frozenset.union(*pd.Series(cities_data).apply(frozenset))

In [32]:
def get_genre_freq(row):
        
    genre_dict = {}
    for genre in row.genres.split(","):
        genre_name, genre_wt = genre.split(":")
        genre_dict[str("genre_" + genre_name)] = int(genre_wt)
    
    genre_freq = {}
    for i in genre_data:
        if i in genre_dict:
            genre_freq[i] = genre_dict[i] 
        else:
            genre_freq[i] = 0            

    return pd.Series(genre_freq)

def get_dow_freq(row):
    
    dow_dict = {}
    for dow in row.dow.split(","):
        dow_name, dow_wt = dow.split(":")
        dow_dict[int(dow_name)] = int(dow_wt)
    
    dow_freq = {}
    for i in dow_data:
        if i in dow_dict:
            dow_freq[i] = dow_dict[i] 
        else:
            dow_freq[i] = 0            

    return pd.Series(dow_freq)

def get_tod_freq(row):
    
    tod_dict = {}
    for tod in row.tod.split(","):
        tod_name, tod_wt = tod.split(":")
        tod_dict[int(tod_name)] = int(tod_wt)
    
    tod_freq = {}
    for i in tod_data:
        if i in tod_dict:
            tod_freq[i] = tod_dict[i] 
        else:
            tod_freq[i] = 0            

    return pd.Series(tod_freq)

In [17]:
genre_feats = train_data.progress_apply(get_genre_freq, axis = 1)
dow_feats = train_data.progress_apply(get_dow_freq, axis = 1)
tod_feats = train_data.progress_apply(get_tod_freq, axis = 1)

train_data = pd.concat([train_data, genre_feats], axis = 1)
train_data = pd.concat([train_data, dow_feats], axis = 1)
train_data = pd.concat([train_data, tod_feats], axis = 1)

In [20]:
genre_feats = test_data.progress_apply(get_genre_freq, axis = 1)
dow_feats = test_data.progress_apply(get_dow_freq, axis = 1)
tod_feats = test_data.progress_apply(get_tod_freq, axis = 1)

test_data = pd.concat([test_data, genre_feats], axis = 1)
test_data = pd.concat([test_data, dow_feats], axis = 1)
test_data = pd.concat([test_data, tod_feats], axis = 1)

100001it [01:00, 1663.43it/s]        
100001it [00:51, 1955.04it/s]        
100001it [00:53, 1867.17it/s]        


In [27]:
# sum of watch time from titles

def get_sum_wt(row):
    
    sum_wt = 0
    
    for title in row.titles.split(","):

        try: #to ignore blanks

            title_name, title_wt = title.split(":")
            sum_wt += int(title_wt)
            
        except:
            pass
        
    return sum_wt

train_data["sum_wt"] = train_data.progress_apply(get_sum_wt, axis = 1)
test_data["sum_wt"] = test_data.progress_apply(get_sum_wt, axis = 1)


  1%|▏         | 1/73 [00:00<00:42,  1.68it/s]
1721it [00:00,  2.39it/s]                     
3645it [00:00,  3.42it/s]
5433it [00:00,  4.88it/s]
7263it [00:00,  6.98it/s]
8887it [00:01,  9.96it/s]
10778it [00:01, 14.23it/s]
12832it [00:01, 20.32it/s]
14559it [00:01, 29.01it/s]
16474it [00:01, 41.42it/s]
18234it [00:01, 59.11it/s]
20169it [00:01, 84.33it/s]
21966it [00:01, 120.23it/s]
23836it [00:01, 171.28it/s]
25641it [00:02, 243.63it/s]
27413it [00:02, 345.88it/s]
29153it [00:02, 489.92it/s]
30919it [00:02, 691.66it/s]
32796it [00:02, 972.72it/s]
35032it [00:02, 1364.16it/s]
36953it [00:02, 1889.62it/s]
38858it [00:02, 2578.07it/s]
40707it [00:02, 3433.03it/s]
42457it [00:02, 4521.32it/s]
44203it [00:03, 5772.81it/s]
45915it [00:03, 6975.98it/s]
47521it [00:03, 8351.78it/s]
49319it [00:03, 9949.37it/s]
51097it [00:03, 11462.89it/s]
52852it [00:03, 12792.65it/s]
54746it [00:03, 14171.35it/s]
56510it [00:03, 14490.78it/s]
58316it [00:03, 15397.23it/s]
60043it [00:04, 15678.66it/s]
61

In [29]:
#Create Count variables
def wcount(p):
    return p.count(',')+1

train_data['title_count'] = train_data['titles'].map(wcount)
train_data['genres_count'] = train_data['genres'].map(wcount)
train_data['cities_count'] = train_data['cities'].map(wcount)
train_data['tod_count'] = train_data['tod'].map(wcount)
train_data['dow_count'] = train_data['dow'].map(wcount)

test_data['title_count'] = test_data['titles'].map(wcount)
test_data['genres_count'] = test_data['genres'].map(wcount)
test_data['cities_count'] = test_data['cities'].map(wcount)
test_data['tod_count'] = test_data['tod'].map(wcount)
test_data['dow_count'] = test_data['dow'].map(wcount)

In [33]:
#Doubtful about how to utilize cities -- too many cities to one-hot encode
train_data.drop(['cities','dow','genres','titles','tod'], inplace=True, axis=1)
test_data.drop(['cities','dow','genres','titles','tod'], inplace=True, axis=1)

In [35]:
train_data.to_csv("train_data.csv", index = False)
test_data.to_csv("test_data.csv", index = False)